In [1]:
import sys
import os
import glob
from pathlib import Path
import random
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd

from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support, roc_curve, roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoConfig
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification, DataCollatorWithPadding


In [2]:
# Configurations
class CFG:
    SEED = 0
    VALID_DATA_PATH = Path('../../radnlp_2024_train_val_20240731/ja/main_task/val')
    MODEL_SAVE_PATH = Path('./model')
    MODEL_NAME = 'debarta'
    MODEL_PATH = 'microsoft/deberta-v3-large'
    # MODEL_NAME = 'globis-university/deberta-v3-japanese-large'
    # MODEL_PATH = 'globis-university/deberta-v3-japanese-large'
    MAX_LENGTH = 512

In [3]:
def get_device() -> str:
    """
    Returns the best available device for PyTorch computations.
    """
    if torch.backends.mps.is_available():
        # macOS with Apple Silicon (MPS backend)
        return "mps"
    elif torch.cuda.is_available():
        # NVIDIA GPU
        return "cuda"
    else:
        # Fallback to CPU
        return "cpu"

try:
    device = torch.device(get_device())
    print(f"Using device: {device}")
except RuntimeError as e:
    print(f"Failed to initialize the device: {e}")
    device = torch.device("cpu")  # Fallback to CPU in case of an error

Using device: cuda


In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False #Trueで高速化重視, Falseで再現性重視
seed_everything(CFG.SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.set_num_threads(1)

In [5]:
def convert_list(text_list, label_dic):
    """
    リストのテキストを、指定された辞書に基づいて数値に変換します。

    Args:
        text_list (list): 変換したいテキストのリスト
        label_dic (dict): テキストと対応する数値の辞書

    Returns:
        list: 数値に変換されたリスト
    """

    num_list = []
    for text in text_list:
        # 辞書から対応する数値を取得
        num = label_dic.get(text, None)
        if num is not None:
            num_list.append(num)
        else:
            print(f"対応する数値が見つかりません: {text}")
    return num_list

label_dic_t = {'T0':0, 'Tis':1, 'T1mi':2, 'T1a':3, 'T1b':4, 'T1c':5, 'T2a':6, 'T2b':7, 'T3':8, 'T4':9}
inverse_label_dic_t = {v: k for k, v in label_dic_t.items()}
label_dic_n = {'N0':0, 'N1':1, 'N2':2, 'N3':3}
inverse_label_dic_n = {v: k for k, v in label_dic_n.items()}
label_dic_m = {'M0':0, 'M1a':1, 'M1b':2, 'M1c':3}
inverse_label_dic_m = {v: k for k, v in label_dic_m.items()}

In [6]:
def create_dataframe(folder_path):
    """
    指定されたフォルダ内のテキストファイルからDataFrameを作成する関数

    Args:
        folder_path (PosixPath): テキストファイルが入っているフォルダのパス

    Returns:
        pandas.DataFrame: 作成されたDataFrame
    """

    # フォルダ内のすべてのテキストファイルのパスを取得
    file_list = list(folder_path.glob('*.txt'))

    # 空のDataFrameを作成
    df_text = pd.DataFrame(columns=['id', 'text'])
    
    # label.csvを読み込む
    df_label = pd.read_csv(os.path.join(folder_path, "label.csv"))

    for file in file_list:
        # ファイル名からIDを抽出
        # file_id = str(file).split("/")[-1].split(".")[0]
        file_id = str(file).split("\\")[-1].split(".")[0] # Windows
        
        # テキストファイルを読み込む
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()

        # DataFrameに追記
        rows = []
        rows.append({'id': int(file_id), 'text': text})
        df_text = pd.concat([df_text, pd.DataFrame(rows)], ignore_index=True)
        
    # textとlabelのDataFrameを結合
    df = pd.merge(df_text, df_label, on='id', how='left')

    return df

val_df = create_dataframe(CFG.VALID_DATA_PATH)

In [7]:
val_df.head(10)

,id,text,t,n,m
0,10365351,左肺中枢側に腫瘤性病変を認めます（103×83×96mm）。原発性肺癌を疑います。左肺の末梢...,T4,N2,M1a
1,10376521,右肺に長径 13.7cm 大、胸壁を超える腫瘤を認めます。T4 と考えます。\n縦隔リンパ節...,T4,N2,M1c
2,10634640,左上葉に 22mm 大の不整形腫瘤影を認め、肺癌が疑われます。\n縦隔に有意なリンパ節腫大は...,T1c,N0,M0
3,10708825,比較可能な画像検査はありません。\n右肺上葉に長径 13.7cm 大の腫瘤影を認めます。腫瘤...,T4,N2,M1c
4,10868892,右下葉に 42mm 大の腫瘤を認め、肺癌を疑います。S6 に同一肺葉内の不連続な副腫\n瘍結...,T3,N2,M1c
5,11407659,右下葉に 42mm 大の腫瘤影を認め、肺癌が疑われます。\n縦隔に腫大リンパ節を認め、転移が...,T2b,N2,M1c
6,11566958,左肺上葉 S3 を中心として 約 3.7cm の 不整形腫瘤性病変を認めます。充実成分を...,T2a,N0,M0
7,11726941,左肺門部に辺縁不整な腫瘤を認めます。ご指摘の癌病変と考えます。\n左主気管支は途絶しており、...,T4,N2,M1a
8,12023275,比較可能な画像検査はありません。\n右肺下葉に長径 42mm 程度の腫瘤影を認め、既知の肺癌...,T2b,N2,M1c
9,12550658,右肺上葉に径 13cm 強の境界明瞭な腫瘤を認めます。癌病変の原発巣と考えます。明\nらかな...,T4,N2,M1c


# 分類モデルの読み込み

In [8]:
tokenizer = AutoTokenizer.from_pretrained(CFG.MODEL_PATH)

base_folder = CFG.MODEL_SAVE_PATH / 't'
trained_model_path = [f for f in base_folder.iterdir() if f.is_dir() and "checkpoint-" in f.name][0]
model_t = AutoModelForSequenceClassification.from_pretrained(trained_model_path)

base_folder = CFG.MODEL_SAVE_PATH / 'n'
trained_model_path = [f for f in base_folder.iterdir() if f.is_dir() and "checkpoint-" in f.name][0]
model_n = AutoModelForSequenceClassification.from_pretrained(trained_model_path)

base_folder = CFG.MODEL_SAVE_PATH / 'm'
trained_model_path = [f for f in base_folder.iterdir() if f.is_dir() and "checkpoint-" in f.name][0]
model_m = AutoModelForSequenceClassification.from_pretrained(trained_model_path)

c:\Users\nishigaki\研究\radnlp\env_radnlp\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
model_t.eval()
model_n.eval()
model_m.eval()

t_pred_li, n_pred_li, m_pred_li = [], [], []

for i in tqdm(range(len(val_df))):
    tokens = tokenizer.encode_plus(val_df.iloc[i]['text'], padding=False, truncation=True, max_length=CFG.MAX_LENGTH, return_tensors="pt")

    with torch.no_grad():
        t_pred = inverse_label_dic_t.get(F.softmax(model_t(**tokens).logits, dim=1).argmax(dim=1).item(), "Unknown")
        n_pred = inverse_label_dic_n.get(F.softmax(model_n(**tokens).logits, dim=1).argmax(dim=1).item(), "Unknown")
        m_pred = inverse_label_dic_m.get(F.softmax(model_m(**tokens).logits, dim=1).argmax(dim=1).item(), "Unknown")

        t_pred_li.append(t_pred)
        n_pred_li.append(n_pred)
        m_pred_li.append(m_pred)
val_df['t_pred'] = t_pred_li
val_df['n_pred'] = n_pred_li
val_df['m_pred'] = m_pred_li

val_df.to_csv('../model_outputs/debarta_results.csv')

  0%|          | 0/54 [00:00<?, ?it/s]

In [10]:
val_df

,id,text,t,n,m,t_pred,n_pred,m_pred
0,10365351,左肺中枢側に腫瘤性病変を認めます（103×83×96mm）。原発性肺癌を疑います。左肺の末梢...,T4,N2,M1a,T4,N2,M1c
1,10376521,右肺に長径 13.7cm 大、胸壁を超える腫瘤を認めます。T4 と考えます。\n縦隔リンパ節...,T4,N2,M1c,T4,N2,M1c
2,10634640,左上葉に 22mm 大の不整形腫瘤影を認め、肺癌が疑われます。\n縦隔に有意なリンパ節腫大は...,T1c,N0,M0,T2b,N0,M0
3,10708825,比較可能な画像検査はありません。\n右肺上葉に長径 13.7cm 大の腫瘤影を認めます。腫瘤...,T4,N2,M1c,T4,N2,M1c
4,10868892,右下葉に 42mm 大の腫瘤を認め、肺癌を疑います。S6 に同一肺葉内の不連続な副腫\n瘍結...,T3,N2,M1c,T3,N2,M1c
5,11407659,右下葉に 42mm 大の腫瘤影を認め、肺癌が疑われます。\n縦隔に腫大リンパ節を認め、転移が...,T2b,N2,M1c,T2b,N2,M1c
6,11566958,左肺上葉 S3 を中心として 約 3.7cm の 不整形腫瘤性病変を認めます。充実成分を...,T2a,N0,M0,T4,N0,M0
7,11726941,左肺門部に辺縁不整な腫瘤を認めます。ご指摘の癌病変と考えます。\n左主気管支は途絶しており、...,T4,N2,M1a,T4,N2,M0
8,12023275,比較可能な画像検査はありません。\n右肺下葉に長径 42mm 程度の腫瘤影を認め、既知の肺癌...,T2b,N2,M1c,T2b,N2,M1c
9,12550658,右肺上葉に径 13cm 強の境界明瞭な腫瘤を認めます。癌病変の原発巣と考えます。明\nらかな...,T4,N2,M1c,T3,N2,M1c


# Submission CSVの出力

In [11]:
sub_df = pd.read_csv('../model_outputs/debarta_results.csv')
t_pred = sub_df['t_pred'].values.tolist()
n_pred = sub_df['n_pred'].values.tolist()
m_pred = sub_df['m_pred'].values.tolist()
t_label = sub_df['t'].values.tolist()
n_label = sub_df['n'].values.tolist()
m_label = sub_df['m'].values.tolist()

print(f't_acc: {accuracy_score(t_label, t_pred)}')
print(f'n_acc: {accuracy_score(n_label, n_pred)}')
print(f'm_acc: {accuracy_score(m_label, m_pred)}')

t_acc: 0.3888888888888889
n_acc: 0.8333333333333334
m_acc: 0.7222222222222222


In [12]:
sub_df = pd.read_csv('../model_outputs/debarta_results.csv')
select_df = sub_df[['id', 't_pred', 'n_pred', 'm_pred']]
rename_df = select_df.rename(columns={'t_pred': 't', 'n_pred': 'n', 'm_pred': 'm'})
rename_df.to_csv('../model_outputs/submission_debarta.csv', index=False)

In [13]:
rename_df.head(10)

,id,t,n,m
0,10365351,T4,N2,M1c
1,10376521,T4,N2,M1c
2,10634640,T2b,N0,M0
3,10708825,T4,N2,M1c
4,10868892,T3,N2,M1c
5,11407659,T2b,N2,M1c
6,11566958,T4,N0,M0
7,11726941,T4,N2,M0
8,12023275,T2b,N2,M1c
9,12550658,T3,N2,M1c


In [14]:
print(rename_df['t'].value_counts())
print(rename_df['n'].value_counts())
print(rename_df['m'].value_counts())

t
T4     25
T2b    12
T3      7
T1c     7
T2a     3
Name: count, dtype: int64
n
N2    29
N0    25
Name: count, dtype: int64
m
M0     28
M1c    26
Name: count, dtype: int64
